<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    SAN Projekt:<br> Schweizer Offshore Firmen
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Florin Barbisch, Gabriel Torres Gamez und Tobias Buess im FS 2024.
</p>

Wir führen eine Voranalyse für das Bundesamt für Statistik durch, um die kürzlich aufgetretenen Leaks aus den Offshore Papers zu untersuchen. 

Diese Analyse zielt darauf ab, Umfang und Natur der Verbindungen in Schweizer Offshore-Strukturen zu ermitteln. Wir verwenden dafür Daten aus der [Offshore Leaks Database](https://offshoreleaks.icij.org/), um mögliche Muster, wichtige Personen aufzudecken, die für die Steuerbehörden oder Regulierungsorgane von Interesse sein könnten. 

Unsere Arbeit umfasst eine detaillierte Prüfung der betroffenen Entitäten. Dies wird es dem Bundesamt für Statistik ermöglichen, fundierte Entscheidungen zur weiteren Untersuchung und möglichen Massnahmen zu treffen.

## Inhalt
1. Wie sehen die Daten aus? Was für Informationen können wir daraus ziehen?
2. Grobe Metriken zu Schweizer Officers und deren Offshore Firmen.
3. ...

In [1]:
import os
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re

# Daten laden

In [3]:
def remove_special_characters(text):
    # Stellen Sie sicher, dass der Text ein String ist
    if isinstance(text, str):
        # Entfernt alles außer Buchstaben, Ziffern, Leerzeichen und grundlegenden Satzzeichen
        return re.sub(r'[^\w\s,.]', '', text)
    return text

nodes_addresses = pd.read_csv("./data/nodes-addresses.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
nodes_addresses["node_type"] = ["Address"]*len(nodes_addresses)

nodes_entities = pd.read_csv("./data/nodes-entities.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
nodes_entities["node_type"] = ["Entity"]*len(nodes_entities)

nodes_intermediaries = pd.read_csv("./data/nodes-intermediaries.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
nodes_intermediaries["node_type"] = ["Intermediary"]*len(nodes_intermediaries)

nodes_officers = pd.read_csv("./data/nodes-officers.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
nodes_officers["node_type"] = ["Officer"]*len(nodes_officers)

nodes_others = pd.read_csv("./data/nodes-others.csv", low_memory=False, index_col=0).astype(str).map(remove_special_characters)
nodes_others["node_type"] = ["Other"]*len(nodes_others)

relationships = pd.read_csv("./data/relationships.csv", low_memory=False).set_index(["node_id_start", "node_id_end"]).astype(str).map(remove_special_characters)

G = nx.MultiDiGraph() #directed multi-edge graph
G.add_nodes_from([(key, value) for key, value in nodes_addresses.to_dict("index").items()])
G.add_nodes_from([(key, value) for key, value in nodes_entities.to_dict("index").items()])
G.add_nodes_from([(key, value) for key, value in nodes_intermediaries.to_dict("index").items()])
G.add_nodes_from([(key, value) for key, value in nodes_officers.to_dict("index").items()])
G.add_nodes_from([(key, value) for key, value in nodes_others.to_dict("index").items()])
G.add_edges_from([(*relationships.index[i], value) for i, value in enumerate(relationships.to_dict(orient='records'))])

del nodes_addresses
del nodes_entities
del nodes_intermediaries
del nodes_officers
del nodes_others
del relationships

graph = G
del G

In [10]:
# get the edge types
edge_types = set()
for edge in graph.edges(data=True):
    # only if node type is Entity or Intermediary
    if graph.nodes[edge[0]]["node_type"] in ["Entity", "Intermediary"] \
    and graph.nodes[edge[1]]["node_type"] in ["Entity", "Intermediary"]:
        edge_types.add(edge[2]["rel_type"])

edge_types = list(edge_types)
edge_types

['similar',
 'same_as',
 'similar_company_as',
 'same_company_as',
 'same_name_as',
 'same_intermediary_as',
 'intermediary_of',
 'officer_of',
 'connected_to',
 'registered_address']

In [13]:
# get all the swiss intermediaries
swiss_intermediaries = []
for node in graph.nodes(data=True):
    if node[1]["node_type"] == "Intermediary" and "Switzerland" in node[1]["countries"]:
        swiss_intermediaries.append(node[0])
print(f"Number of Swiss intermediaries: {len(swiss_intermediaries)}")

Number of Swiss intermediaries: 1332


In [58]:
# create a new set of edges that connect the swiss intermediaries
# give them a weight and increase it with every entity they are connected to

# all edges point from an intermediary to an entity
swiss_intermediary_edges = {} # key: entity, value: {swiss_intermediary: weight}
for intermediary in swiss_intermediaries:
    edges = graph.edges(intermediary, data=True)
    if len(edges) < 1:
        continue
    for edge in edges:
        other_node = graph.nodes[edge[1]]
        if other_node["node_type"] == "Entity":
            if edge[1] in swiss_intermediary_edges:
                swiss_intermediary_edges[edge[1]][intermediary] = swiss_intermediary_edges[edge[1]].get(intermediary, 0) + 1
            else:
                swiss_intermediary_edges[edge[1]] = {intermediary: 1}
n_intermediaries = [len(x) for x in swiss_intermediary_edges.values() if len(x) > 1]
print(f"Number of entities with more than one swiss intermediary: {len(n_intermediaries)}")

Number of entities with more than one swiss intermediary: 0


Fazit: ein Entity hat maximal 1 Intermediary. Intermediaries können also nicht über mehrere Entities verknüpft werden.

Wir können also noch versuchen die Intermediaries über die Officers zu verknüpfen. Dazu macht es aber Sinn, zuerst herauszufinden, ob Officers<>Entities oder Officers<>Intermediaries mehr Verbindungen haben könnten.

In [63]:
def get_cardinality(graph, node_type_1, node_type_2):
    """
    Get the cardinality of node_type_1 to node_type_2

    Parameters:
    graph: nx.MultiDiGraph
    node_type_1: str
    node_type_2: str

    Returns:
    int: number of connections
    int: number of nodes with more than one connection
    """
    cardinality = {}
    for node in graph.nodes(data=True):
        if node[1]["node_type"] == node_type_1:
            edges = graph.edges(node[0], data=True)
            if len(edges) < 1:
                continue
            for edge in edges:
                other_node = graph.nodes[edge[1]]
                if other_node["node_type"] == node_type_2:
                    if node[0] in cardinality:
                        cardinality[node[0]].append(edge[1])
                    else:
                        cardinality[node[0]] = [edge[1]]
    return len(cardinality), len([len(x) for x in cardinality.values() if len(x) > 1])
# get the cardinality of officers to entities
n_connections, n_multiple_connections = get_cardinality(graph, "Officer", "Entity")
print(f"Number of connections between officers and entities: {n_connections}, number of officers with more than one entity: {n_multiple_connections}")

Number of connections between officers and entities: 770259, number of officers with more than one entity: 198175


In [129]:
node_types = set()
longest_path = 0
swiss_intermediary_edges = {} # key: (intermediary_1, intermediary_2), value: [distance]

graph_undirected = graph.to_undirected(as_view=True)

for i, intermediary in enumerate(swiss_intermediaries):
    print(f"Intermediary {i+1}/{len(swiss_intermediaries)}", len(swiss_intermediary_edges), end="\r")
    shortest_paths = {}
    for other_intermediary in swiss_intermediaries:
        if other_intermediary < intermediary:
            try:
                shortest_paths[other_intermediary] = nx.shortest_path(graph_undirected, intermediary, other_intermediary)
            except nx.NetworkXNoPath:
                pass
    for node, path in shortest_paths.items():
        if len(path) > longest_path:
            longest_path = len(path)
        for node in path:
            node_types.add(graph.nodes[node]["node_type"])
        if node in swiss_intermediaries and node != intermediary:
            if node < intermediary:
                key = (node, intermediary)
            else:
                key = (intermediary, node)
                print("ALAARM!!!!!")
            swiss_intermediary_edges[key] = swiss_intermediary_edges.get(key, []) + [len(path)-1]

print(f"Longest path from swiss intermediary to entity: {longest_path}")
print(f"Node types in shortest path: {node_types}")
print(f"Number of intermediary pairs with multiple paths: {len([edge for edge in swiss_intermediary_edges.keys() if len(swiss_intermediary_edges[edge]) > 1])}")

In [128]:
len(swiss_intermediary_edges)

6750

In [104]:
list(swiss_intermediary_edges.items())[:5]

[((11000007, 11000007), [0]),
 ((11000010, 11000010), [0]),
 ((11000016, 11000016), [0]),
 ((11000018, 11000018), [0]),
 ((11000019, 11000019), [0])]

In [91]:
# longest path of 5 validates that cutoff=10 is sufficient
# the Number of intermediary pairs with multiple paths: 0 validates that we only have shortest path between intermediaries

# create a new graph with the swiss intermediaries
swiss_intermediaries_graph = nx.Graph()
swiss_intermediaries_graph.add_nodes_from([(node, graph.nodes[node]) for node in swiss_intermediaries])
swiss_intermediaries_graph.add_edges_from([(key[0], key[1], {"weight": np.mean(value)}) for key, value in swiss_intermediary_edges.items()])

In [90]:
swiss_intermediaries_graph.add_nodes_from([(node, graph.nodes[node]) for node in swiss_intermediaries])
# add 
# get a random node
node = swiss_intermediaries[0]

print(f"Node {node}: {swiss_intermediaries_graph.nodes[node]}")

Node 11000007: {'name': 'FIGEST CONSEIL S.A.', 'status': 'ACTIVE', 'internal_id': '10064', 'address': 'nan', 'countries': 'Switzerland', 'country_codes': 'CHE', 'sourceID': 'Panama Papers', 'valid_until': 'The Panama Papers  data is current through 2015', 'note': 'nan', 'node_type': 'Intermediary'}


In [94]:
swiss_intermediaries_graph.number_of_edges(), swiss_intermediaries_graph.number_of_nodes()

(1332, 1332)

In [100]:
# get a list with the degree of each node
degrees = [degree for node, degree in swiss_intermediaries_graph.degree()]
degrees_sorted = sorted(degrees, reverse=True)
print(f"Top 5 degrees: {degrees_sorted[:5]}")
print(f"Bottom 5 degrees: {degrees_sorted[-5:]}")

Top 5 degrees: [2, 2, 2, 2, 2]
Bottom 5 degrees: [2, 2, 2, 2, 2]


In [64]:
n_connections, n_multiple_connections = get_cardinality(graph, "Entity", "Officer")
print(f"Number of connections between entities and officers: {n_connections}, number of entities with more than one officer: {n_multiple_connections}")

Number of connections between entities and officers: 605, number of entities with more than one officer: 10


## Wie sehen die Daten aus? Was für Informationen können wir daraus ziehen?

## Grobe Metriken zu schweizer Officers und deren Offshore Firmen.

## ...